# Multi-tenant Chat with Papers - Setup
## Get keys and urls

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

WEAVIATE_URL = os.getenv("WEAVIATE_URL")
WEAVIATE_KEY = os.getenv("WEAVIATE_KEY")

print(WEAVIATE_URL)
print(WEAVIATE_KEY)

## Connect to Weaviate

In [ ]:
import weaviate
from weaviate.classes.init import Auth
# from weaviate.classes.init import AdditionalConfig, Timeout

client = weaviate.connect_to_weaviate_cloud(
    cluster_url=WEAVIATE_URL,
    auth_credentials=Auth.api_key(WEAVIATE_KEY),

    # additional_config=AdditionalConfig(
    #     timeout=Timeout(init=2, query=45, insert=120),  # Values in seconds
    # )
)

client.is_ready()

## Create Tenant-ready collection

In [ ]:
from weaviate.classes.config import Configure

if (client.collections.exists("Papers")):
    client.collections.delete("Papers")

client.collections.create(
    "Papers",

    vectorizer_config=[
        Configure.NamedVectors.text2vec_weaviate(
            name="main_vector",

            model="Snowflake/snowflake-arctic-embed-l-v2.0",
            source_properties=["chunk"]
        )
    ],

    # supported models: https://weaviate.io/developers/weaviate/model-providers/openai/generative#available-models
    generative_config=Configure.Generative.openai(
        model="gpt-4" # gpt-3.5-turbo
    ),

    multi_tenancy_config=Configure.multi_tenancy(True)
)

## List Tenants

In [ ]:
papers = client.collections.get("Papers")
papers.tenants.get()

## Close the client

In [ ]:
client.close()